<a href="https://colab.research.google.com/github/kobrue02/BENALI/blob/main/notebooks/llama_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from huggingface_hub import login
login()

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize model and tokenizer
checkpoint = "swiss-ai/Apertus-8B-Instruct-2509"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to("cuda")

def get_l1_prediction(text, l1s):
    labels = "\n- ".join(l1s)
    prompt = """Consider the following text which was written by a non-native speaker:
    `{text}`
    \nWhat do you think is their native language?
    Choose one of the following:
    {labels}
    Return only the answer.
    The native language is: """.format(text=text, labels=l1s)

    messages = [
        {"role": "user", "content": prompt},
    ]

    # Apply chat template and tokenize
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode only the new tokens (response part)
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    #print(f"Raw response: {response}")

    # Parse the response
    if "The native language is: " in response:
        result = response.split("The native language is: ")[1].strip()
        # Clean up common issues
        result = result.split('\n')[0].split('.')[0].strip()
        # remove leading/trailing quotations if any
        if result.startswith('"') and result.endswith('"'):
            result = result[1:-1]
        if result.startswith("'") and result.endswith("'"):
            result = result[1:-1]
        return result
    else:
        # Fallback parsing - look for any of the labels in the response
        response_lower = response.lower()
        for label in l1s:
            if label.lower() in response_lower:
                return label
        return "Unknown"

tokenizer_config.json:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/901 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

CUDA-fused xIELU not available (No module named 'xielu') – falling back to a Python version.
For CUDA xIELU (experimental), `pip install git+https://github.com/nickjbrowning/XIELU`


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [7]:
import pandas as pd

test_df = pd.read_csv("nli_test.csv")
test_df.head()

,sentence,l1
0,It would be moree accurate to say Athens (3 mi...,Greece
1,#编号：540\n#篇名：兄弟射雁\n#字数／字符数：411/411\n#类型：习作\n#姓...,Vietnam
2,"Not that they are poor, they just don't give m...",Portugal
3,Red in the corners with white surrounding them,Norway
4,>Wenn es stets zu Schutz und Trutze,Germany


In [8]:
test_sample = test_df.iloc[0]
test_sample

,0
sentence,It would be moree accurate to say Athens (3 mi...
l1,Greece


In [10]:
model_response = get_l1_prediction(test_sample["sentence"], test_df["l1"].unique())
model_response

'Portugal'

In [ ]:
# on the full test set
from tqdm import tqdm
from transformers import logging as transformers_logging
import warnings

transformers_logging.set_verbosity_error()
warnings.filterwarnings("ignore")
tqdm.pandas()
labels = test_df["l1"].unique()
test_sample = test_df.sample(5000)
test_sample["l1_pred"] = test_sample.progress_apply(lambda x: get_l1_prediction(x["sentence"], labels), axis=1)

 53%|█████▎    | 2667/5000 [45:22<35:19,  1.10it/s]

In [1]:
import torch
torch.cuda.empty_cache()

# also run garbage collection
import gc
gc.collect()

30

In [19]:
test_sample.to_csv("llama_predictions_sample.csv", index=False)

In [2]:
# classification report on the l1 predictions
from sklearn.metrics import classification_report

print(classification_report(test_sample["l1"], test_sample["l1_pred"]))

NameError: name 'test_sample' is not defined